# 🧜‍♂️ MermaidTrace Interactive Online Demo

Welcome to **MermaidTrace**! A powerful Python logic visualization tool that automatically transforms your code execution flow into clear Mermaid sequence diagrams.

In this notebook, you can experience the core features of MermaidTrace without any local setup:
- ✅ **Automation**: Just use decorators, no manual drawing needed.
- ✅ **Deep Tracing**: Supports classes, nested functions, and async calls.
- ✅ **Error Visualization**: Automatically captures and visualizes exactly where errors occur.
- ✅ **Intelligent Collapsing**: Handles loops and repetitive calls gracefully.

## 1. Installation

First, let's install the `mermaid-trace` library.

In [ ]:
!pip install mermaid-trace

## 2. Setup Visualization Helper

To see the generated diagrams directly in Colab, we'll define a simple helper function to render Mermaid code.

In [ ]:
from IPython.display import display, HTML
import uuid

def render_mermaid(filename):
    with open(filename, "r") as f:
        content = f.read()
    
    # Generate a unique ID for each diagram to avoid conflicts
    div_id = f"mermaid-{uuid.uuid4().hex[:8]}"
    
    # Render using Mermaid.js (v10+ recommended way)
    html = f"""
    <div id=\"{div_id}\" class=\"mermaid\">
    {content}
    </div>
    <script type=\"module\">
        import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
        
        // Initialize configuration
        mermaid.initialize({{ startOnLoad: false, theme: 'default' }});
        
        // Explicitly trigger rendering for the specific element
        const element = document.getElementById('{div_id}');
        if (element) {{
            await mermaid.run({{
                nodes: [element]
            }});
        }}
    </script>
    """
    display(HTML(html))

## 3. Basic Usage: Simple Business Flow

Imagine an order process: User Login -> Check Inventory -> Process Payment.

In [ ]:
from mermaid_trace import trace, configure_flow

# Configure output file
configure_flow("basic_demo.mmd", overwrite=True)

@trace(source="User", target="AuthService")
def login(username):
    return verify_db(username)

@trace(source="AuthService", target="Database")
def verify_db(username):
    return True

@trace(source="User", target="OrderSys")
def place_order(item_id):
    if login("admin"):
        check_stock(item_id)
        process_payment()

@trace(source="OrderSys", target="Inventory")
def check_stock(item_id):
    return True

@trace(source="OrderSys", target="PayGateway")
def process_payment():
    return "Success"

# Run the logic
place_order("iphone_15")

print("✅ Diagram generated! Rendering...")
render_mermaid("basic_demo.mmd")

## 4. Advanced Usage: Class Method Tracing

For complex classes, you don't need to add decorators to every method manually. Use `@trace_class` instead.

In [ ]:
from mermaid_trace import trace_class

configure_flow("class_demo.mmd", overwrite=True)

@trace_class(target="PaymentService")
class PaymentProcessor:
    def validate_card(self, card_no):
        return True
    
    def charge(self, amount):
        self.validate_card("1234")
        return "Paid"

processor = PaymentProcessor()
processor.charge(100)

render_mermaid("class_demo.mmd")

## 5. Error Visualization

When your code crashes, MermaidTrace automatically captures and highlights the error path in the diagram.

In [ ]:
configure_flow("error_demo.mmd", overwrite=True)

@trace(source="Client", target="API")
def api_call():
    dangerous_operation()

@trace(source="API", target="Server")
def dangerous_operation():
    raise ValueError("Database Connection Timeout!")

try:
    api_call()
except Exception as e:
    print(f"Caught expected error: {e}")

render_mermaid("error_demo.mmd")

## 6. Async Tracing

MermaidTrace has first-class support for `asyncio` logic.

In [ ]:
import asyncio
from mermaid_trace import trace

configure_flow("async_demo.mmd", overwrite=True)

@trace(source="User", target="AsyncServer")
async def async_task():
    await asyncio.sleep(0.1)
    await sub_task()

@trace(source="AsyncServer", target="DB")
async def sub_task():
    return "Done"

# Run async function directly in Colab
await async_task()

render_mermaid("async_demo.mmd")

## 7. Conclusion

As you can see, MermaidTrace makes hidden logic visible and easy to debug.

If you find it useful, please:
- ⭐ Star it on GitHub: [xt765/mermaid-trace](https://github.com/xt765/mermaid-trace)
- 📖 Read the full docs: [User Guide](https://github.com/xt765/mermaid-trace/blob/main/docs/en/USER_GUIDE.md)